In [1]:
import requests
import zipfile
import pandas as pd
from contextlib import closing
import io

In [1]:
save_directory = '../../../Data_Sets/citibike/'

def extract_clean(save_directory, file_list):
    header_format1 = {
        'bike_id':'ride_id',
        'starttime':'started_at',
        'stoptime':'ended_at',
        'start station name':'start_station_name',
        'start station id':'start_staton_id',
        'end station name':'end_station_name',
        'end station id':'end_station_id',
        'start station latitude':'start_lat',
        'start station longitude':'start_lng',
        'end station latitude':'end_lat',
        'end station longitude':'end_lng',
        'usertype':'member_casual',
        'birth year':'Birth Year',
        'gender':'Gender',
        'trip duration':'Trip Duration'
        }
    header_format2 = {
        'Bike ID':'ride_id',
        'Start Time':'started_at',
        'Stoptime':'ended_at',
        'Start Station Name':'start_station_name',
        'Start Station ID':'start_staton_id',
        'End Station Name':'end_station_name',
        'End Station ID':'end_station_id',
        'Start Station Latitude':'start_lat',
        'Start Station Longitude':'start_lng',
        'End Station Latitude':'end_lat',
        'End Station Longitude':'end_lng',
        'User Type':'member_casual',
        'Birth Year':'Birth Year',
        'Gender':'Gender',
        'trip duration':'Trip Duration'
        }
    final_df = pd.DataFrame()
    
    for file in file_list:
        url1 = 'https://s3.amazonaws.com/tripdata/' + file + '.zip'
        url2 = 'https://s3.amazonaws.com/tripdata/' + file + '.csv.zip'
        csv_path =  save_directory + file + '.csv'
        
        #attempt to locate the specified file
        response = requests.get(url1)
        if response.status_code != 200:
            response = requests.get(url2)
            if response.status_code != 200:
                print(f'{file} is unavailable')
                continue

        # rename the column headers and replace entries to match the 2021 format, at the end save the data as a csv file
        with closing(response), zipfile.ZipFile(io.BytesIO(response.content)) as zip_contents:
            file_list = zip_contents.namelist()
            with zip_contents.open(file_list[0]) as tempfile:
                bike_df = pd.read_csv(tempfile)
                if 'usertype' in bike_df.columns:
                    bike_df = bike_df.rename(columns=header_format1)     
                elif 'User Type' in bike_df.columns:
                    bike_df = bike_df.rename(columns=header_format2)
                elif 'member_casual' in bike_df.columns:
                    pass
                else:
                    print(f'{file} has an unknown format, unable to clean')
                    continue
                bike_df['member_casual'] = bike_df['member_casual'].replace({
                    'Subscriber':'member',
                    'Customer':'casual'
                })
                final_df = pd.concat([final_df,bike_df])
            tempfile.close()
        print(f'Successfully extracted and cleaned {file}')
        
    final_df.to_csv(save_directory + '202106-201306-citibike-tripdata.csv')
    print('Final CSV file successfully saved!')

In [3]:
# generate list of file names to download and clean
years = ['2013']
months = ['01','02','03','04',
          '05','06','07','08',
          '09','10','11','12']

file_names = [f'{year}{month}-citibike-tripdata' for year in years for month in months]
file_names

['201301-citibike-tripdata',
 '201302-citibike-tripdata',
 '201303-citibike-tripdata',
 '201304-citibike-tripdata',
 '201305-citibike-tripdata',
 '201306-citibike-tripdata',
 '201307-citibike-tripdata',
 '201308-citibike-tripdata',
 '201309-citibike-tripdata',
 '201310-citibike-tripdata',
 '201311-citibike-tripdata',
 '201312-citibike-tripdata',
 '201401-citibike-tripdata',
 '201402-citibike-tripdata',
 '201403-citibike-tripdata',
 '201404-citibike-tripdata',
 '201405-citibike-tripdata',
 '201406-citibike-tripdata',
 '201407-citibike-tripdata',
 '201408-citibike-tripdata',
 '201409-citibike-tripdata',
 '201410-citibike-tripdata',
 '201411-citibike-tripdata',
 '201412-citibike-tripdata',
 '201501-citibike-tripdata',
 '201502-citibike-tripdata',
 '201503-citibike-tripdata',
 '201504-citibike-tripdata',
 '201505-citibike-tripdata',
 '201506-citibike-tripdata',
 '201507-citibike-tripdata',
 '201508-citibike-tripdata',
 '201509-citibike-tripdata',
 '201510-citibike-tripdata',
 '201511-citib

In [5]:
file = '202104-citibike-tripdata'
url1 = 'https://s3.amazonaws.com/tripdata/' + file + '.zip'
url2 = 'https://s3.amazonaws.com/tripdata/' + file + '.csv.zip'


#attempt to locate the specified file
response = requests.get(url1)
if response.status_code != 200:
    response = requests.get(url2)
    if response.status_code != 200:
        print(f'{file} is unavailable')
test_df = pd.DataFrame()
# rename the column headers and replace entries to match the 2021 format, at the end save the data as a csv file
with closing(response), zipfile.ZipFile(io.BytesIO(response.content)) as zip_contents:
    file_list = zip_contents.namelist()
    with zip_contents.open(file_list[0]) as tempfile:
        bike_df = pd.read_csv(tempfile)
        if 'usertype' in bike_df.columns:
            bike_df = bike_df.rename(columns=header_format1)     
        elif 'User Type' in bike_df.columns:
            bike_df = bike_df.rename(columns=header_format2)
        elif 'member_casual' in bike_df.columns:
            pass
        else:
            print(f'{file} has an unknown format, unable to clean')
        bike_df['member_casual'] = bike_df['member_casual'].replace({'Subscriber':'member',
                                                                     'Customer':'casual'
                                                                    })
        test_df = bike_df
    

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
display(test_df)
display(test_df.dtypes)
display(test_df.memory_usage())

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,7099F9531C9DA16D,docked_bike,2021-04-28 17:27:24,2021-04-28 17:34:31,DeKalb Ave & Hudson Ave,4513.06,Douglass St & 4 Ave,4175.14,40.689888,-73.981013,40.679279,-73.981540,member
1,EDC6F1B1A69A7863,docked_bike,2021-04-27 17:28:02,2021-04-27 17:33:07,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member
2,431A76830E10408A,docked_bike,2021-04-28 18:05:19,2021-04-28 18:10:48,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member
3,C00248A0ADC009FA,docked_bike,2021-04-20 21:35:47,2021-04-20 21:56:31,Irving Ave & DeKalb Ave,4898.02,Division Ave & Hooper St,5045.05,40.702700,-73.920950,40.706842,-73.954435,casual
4,BE78810D746EE516,docked_bike,2021-04-30 00:34:54,2021-04-30 01:03:29,3 Ave & E 112 St,7543.15,E 115 St & Lexington Ave,7599.09,40.795508,-73.941606,40.797911,-73.942300,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067664,DE4F31703A6FBD72,docked_bike,2021-04-18 15:48:21,2021-04-18 16:03:42,Hudson St & W 13 St,6115.06,Mercer St & Spring St,5532.01,40.740057,-74.005274,40.723627,-73.999496,casual
2067665,391040FC8CAF2601,docked_bike,2021-04-19 19:39:00,2021-04-19 19:55:18,W 87 St & Amsterdam Ave,7458.03,3 Ave & E 62 St,6762.04,40.788390,-73.974700,40.763126,-73.965269,member
2067666,1DB5175D0F873C53,docked_bike,2021-04-21 11:57:03,2021-04-21 12:15:49,E 27 St & 1 Ave,6004.06,3 Ave & E 62 St,6762.04,40.739445,-73.976806,40.763126,-73.965269,member
2067667,3004043A978641B3,docked_bike,2021-04-04 14:41:23,2021-04-04 14:56:46,Myrtle Ave & St Edwards St,4659.02,Brooklyn Bridge Park - Pier 2,4756.04,40.693270,-73.977038,40.698458,-73.997178,member


ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

Index                      128
ride_id               16541352
rideable_type         16541352
started_at            16541352
ended_at              16541352
start_station_name    16541352
start_station_id      16541352
end_station_name      16541352
end_station_id        16541352
start_lat             16541352
start_lng             16541352
end_lat               16541352
end_lng               16541352
member_casual         16541352
dtype: int64

In [12]:
test_df = test_df.astype('category')

In [26]:
display(test_df)
display(test_df.dtypes)
display(test_df.memory_usage())

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,7099F9531C9DA16D,docked_bike,2021-04-28 17:27:24,2021-04-28 17:34:31,DeKalb Ave & Hudson Ave,4513.06,Douglass St & 4 Ave,4175.14,40.689888,-73.981013,40.6792788,-73.98154004,member
1,EDC6F1B1A69A7863,docked_bike,2021-04-27 17:28:02,2021-04-27 17:33:07,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.76132983124814,-73.97982001304626,member
2,431A76830E10408A,docked_bike,2021-04-28 18:05:19,2021-04-28 18:10:48,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.76132983124814,-73.97982001304626,member
3,C00248A0ADC009FA,docked_bike,2021-04-20 21:35:47,2021-04-20 21:56:31,Irving Ave & DeKalb Ave,4898.02,Division Ave & Hooper St,5045.05,40.7027,-73.92095,40.70684203101325,-73.9544353965357,casual
4,BE78810D746EE516,docked_bike,2021-04-30 00:34:54,2021-04-30 01:03:29,3 Ave & E 112 St,7543.15,E 115 St & Lexington Ave,7599.09,40.795508,-73.941606,40.797911,-73.9423,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067664,DE4F31703A6FBD72,docked_bike,2021-04-18 15:48:21,2021-04-18 16:03:42,Hudson St & W 13 St,6115.06,Mercer St & Spring St,5532.01,40.740057,-74.005274,40.72362738,-73.99949601,casual
2067665,391040FC8CAF2601,docked_bike,2021-04-19 19:39:00,2021-04-19 19:55:18,W 87 St & Amsterdam Ave,7458.03,3 Ave & E 62 St,6762.04,40.78839,-73.9747,40.76312584,-73.96526895,member
2067666,1DB5175D0F873C53,docked_bike,2021-04-21 11:57:03,2021-04-21 12:15:49,E 27 St & 1 Ave,6004.06,3 Ave & E 62 St,6762.04,40.739445,-73.976806,40.76312584,-73.96526895,member
2067667,3004043A978641B3,docked_bike,2021-04-04 14:41:23,2021-04-04 14:56:46,Myrtle Ave & St Edwards St,4659.02,Brooklyn Bridge Park - Pier 2,4756.04,40.69327,-73.977038,40.698458,-73.997178,member


ride_id                 string
rideable_type         category
started_at              string
ended_at                string
start_station_name    category
start_station_id      category
end_station_name      category
end_station_id        category
start_lat             category
start_lng             category
end_lat               category
end_lng               category
member_casual         category
dtype: object

Index                      128
ride_id               16541352
rideable_type          2067793
started_at            16541352
ended_at              16541352
start_station_name     4179562
start_station_id       4180082
end_station_name       4179738
end_station_id         4180098
start_lat              4215282
start_lng              4215266
end_lat                4218082
end_lng                4217994
member_casual          2067793
dtype: int64

In [31]:
category_list = [
    'rideable_type','start_station_name','start_station_id',
    'end_station_name','end_station_id','start_lat',
    'start_lng','end_lat','end_lng','member_casual']
test_df[category_list] = test_df[category_list].astype('category')

In [25]:
display(test_df)
display(test_df.dtypes
display(test_df.memory_usage())

SyntaxError: invalid syntax (<ipython-input-25-0db91c1d3b3d>, line 3)

In [16]:
test_df = test_df.astype('string')

In [28]:
display(test_df)
display(test_df.dtypes)
display(test_df.memory_usage().sum() / (1024**2))

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,7099F9531C9DA16D,docked_bike,2021-04-28 17:27:24,2021-04-28 17:34:31,DeKalb Ave & Hudson Ave,4513.06,Douglass St & 4 Ave,4175.14,40.689888,-73.981013,40.6792788,-73.98154004,member
1,EDC6F1B1A69A7863,docked_bike,2021-04-27 17:28:02,2021-04-27 17:33:07,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.76132983124814,-73.97982001304626,member
2,431A76830E10408A,docked_bike,2021-04-28 18:05:19,2021-04-28 18:10:48,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.76132983124814,-73.97982001304626,member
3,C00248A0ADC009FA,docked_bike,2021-04-20 21:35:47,2021-04-20 21:56:31,Irving Ave & DeKalb Ave,4898.02,Division Ave & Hooper St,5045.05,40.7027,-73.92095,40.70684203101325,-73.9544353965357,casual
4,BE78810D746EE516,docked_bike,2021-04-30 00:34:54,2021-04-30 01:03:29,3 Ave & E 112 St,7543.15,E 115 St & Lexington Ave,7599.09,40.795508,-73.941606,40.797911,-73.9423,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067664,DE4F31703A6FBD72,docked_bike,2021-04-18 15:48:21,2021-04-18 16:03:42,Hudson St & W 13 St,6115.06,Mercer St & Spring St,5532.01,40.740057,-74.005274,40.72362738,-73.99949601,casual
2067665,391040FC8CAF2601,docked_bike,2021-04-19 19:39:00,2021-04-19 19:55:18,W 87 St & Amsterdam Ave,7458.03,3 Ave & E 62 St,6762.04,40.78839,-73.9747,40.76312584,-73.96526895,member
2067666,1DB5175D0F873C53,docked_bike,2021-04-21 11:57:03,2021-04-21 12:15:49,E 27 St & 1 Ave,6004.06,3 Ave & E 62 St,6762.04,40.739445,-73.976806,40.76312584,-73.96526895,member
2067667,3004043A978641B3,docked_bike,2021-04-04 14:41:23,2021-04-04 14:56:46,Myrtle Ave & St Edwards St,4659.02,Brooklyn Bridge Park - Pier 2,4756.04,40.69327,-73.977038,40.698458,-73.997178,member


ride_id                 string
rideable_type         category
started_at              string
ended_at                string
start_station_name    category
start_station_id      category
end_station_name      category
end_station_id        category
start_lat             category
start_lng             category
end_lat               category
end_lng               category
member_casual         category
dtype: object

83.29951667785645